In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import flask

In [3]:
!pip install flask-ngrok

In [4]:
from flask import Flask,render_template,request,flash
import os
from flask_ngrok import run_with_ngrok

In [5]:
app=Flask(__name__,template_folder='/', static_folder='/content/drive/MyDrive/Caption Generator/flask image caption app/static')
run_with_ngrok(app)
app.config["SEND_FILE_MAX_AGE_DEFAULT"]=1

from keras.models import load_model
import numpy as np
from keras.applications import ResNet50
from keras.optimizers import Adam
from keras.layers import Dense, Flatten,Input, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from keras.models import Sequential, Model,load_model
from keras.utils import np_utils
from keras.preprocessing import image, sequence
import cv2
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import matplotlib.pyplot as plt

resnet=ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')
print("resnet loaded")
embedding_size = 128

max_len = 36
vocab=np.load("/content/drive/MyDrive/Caption Generator/caption generator/vocab.npy",allow_pickle=True)
vocab=vocab.item()
inv_vocab={v:k for k,v in vocab.items()}
vocab_size = len(vocab)
model=load_model('/content/drive/MyDrive/Caption Generator/caption generator/model.h5')
model.load_weights("/content/drive/MyDrive/Caption Generator/caption generator/modelweights.h5")
print("model loaded")

@app.route('/')
def index():
  return render_template('/content/drive/MyDrive/Caption Generator/flask image caption app/static/index.html')

@app.route('/next',methods=['POST','GET'])
def predict():
  global model,vocab,inv_vocab,resnet
  file = request.files['img']
  file.save("/content/drive/MyDrive/Caption Generator/flask image caption app/static/img.jpg")
  img=cv2.imread("/content/drive/MyDrive/Caption Generator/flask image caption app/static/img.jpg")
  img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  plt.imshow(img)
  img= cv2.resize(img,(224,224))
  img=np.reshape(img,(1,224,224,3))
  features=resnet.predict(img).reshape(1,2048)
  text_in=['startofseq']
  final=""
  print("*"*50)
  print("getting caption")

  count=0
  while tqdm(count < 20):
      count += 1

      encoded = []
      for i in text_in:
          encoded.append(vocab[i])

      padding= pad_sequences([encoded], padding='post', truncating='post', maxlen=max_len)


      prediction = np.argmax(model.predict([features, padding]))

      sampled_word = inv_vocab[prediction]
          
      if (sampled_word == '.' or sampled_word =='endofseq'):
        break
      else:
        final=final+" "+sampled_word

      text_in.append(sampled_word)
  
  return render_template("/content/drive/MyDrive/Caption Generator/flask image caption app/static/predict.html",final=final)

94773248/94765736 [==============================] - 1s 0us/step
resnet loaded
model loaded


In [ ]:
if __name__ == '__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c7068f527ec8.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/May/2021 03:36:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 03:36:32] "GET /css/main.css HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 03:36:32] "GET /js/jquery.min.js HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 03:36:33] "GET /js/jquery.scrolly.min.js HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 03:36:33] "GET /js/browser.min.js HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 03:36:34] "GET /js/breakpoints.min.js HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 03:36:34] "GET /js/util.js HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 03:36:34] "GET /js/main.js HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 03:36:34] "GET /favicon.ico HTTP/1.1" 404 -
0it [00:00, ?it/s]

**************************************************
getting caption



0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
127.0.0.1 - - [04/May/2021 03:37:27] "POST /next HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 03:37:27] "GET /css/main.css HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 03:37:27] "GET /static/img.jpg HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 03:37:28] "GET /js/jquery.min.js HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 03:37:28] "GET /js/jquery.scrolly.min.js HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 03:37:28] "GET /js/browser.min.js HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 03:37:28] "GET /js/breakpoints.min.js HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 03:37:28] "GET /js/main.js HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 03:37:28] "GET /js/util.js HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 03:37:29] "GET /js/jquery.scrolly.min.js HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 03:37:29] 